In [24]:
import numpy as np
import plotly.graph_objects as go

In [25]:
def matlab_script():
    # Constants
    TIMESTEP = 0.01
    NUMSAMPLES = 1000
    MOD_FREQ_HZ = 40
    CHANNEL_SEPARATION_HZ = 40

    # Main script execution
    signal = generate_signal(TIMESTEP, NUMSAMPLES)
    visualize_signal_and_fft_simpsons(signal, TIMESTEP, MOD_FREQ_HZ, CHANNEL_SEPARATION_HZ)

In [26]:
def generate_signal(timestep, numsamples):
    t = np.linspace(0, numsamples*timestep, numsamples)
    windowed_signal = np.sin(40.0 * 2.0 * np.pi * t) * np.hamming(numsamples)
    return windowed_signal

def fft_calculate(data, timestep):
    yf = np.abs(np.fft.fft(data))
    numsamples = len(data)
    freq = np.fft.fftfreq(numsamples, d=timestep)
    xf = freq[:numsamples//2]
    yf = yf[:numsamples//2] * 2.0 / numsamples
    return xf, yf

def find_nearest(array, value):
    idx = np.argmin(np.abs(array - value))
    nearestValue = array[idx]
    return idx, nearestValue

In [27]:
def simpsons_integration(xf, yf, idx_start, idx_stop):
    n = idx_stop - idx_start
    if n % 2 == 0:
        raise ValueError("Number of intervals should be even for Simpson's rule.")
    
    h = (xf[idx_stop] - xf[idx_start]) / n
    result = 0

    for i in range(n + 1):
        y = yf[idx_start + i]
        if i == 0 or i == n:
            result += y
        elif i % 2 == 0:
            result += 2*y
        else:
            result += 4*y

    result *= h/3
    return result

In [28]:
def visualize_signal_and_fft_simpsons(signal, timestep, mod_freq_hz, channel_separation_hz):
    xf, yf = fft_calculate(signal, timestep)

    freq_start = mod_freq_hz - channel_separation_hz / 2
    freq_stop = mod_freq_hz + channel_separation_hz / 2

    idx_start, _ = find_nearest(xf, freq_start)
    idx_stop, _ = find_nearest(xf, freq_stop)

    # Ensure even number of intervals
    if (idx_stop - idx_start) % 2 == 0:
        idx_stop += 1

    integrated_area = simpsons_integration(xf, yf, idx_start, idx_stop)

    # Time-domain Signal plot
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(y=signal, mode='lines', name='Signal'))
    fig1.update_layout(title='Time-domain Signal')
    fig1.show()

    # FFT Magnitude plot
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=xf, y=yf, mode='lines', name='FFT'))
    fig2.add_trace(go.Scatter(x=xf[idx_start:idx_stop+1], y=yf[idx_start:idx_stop+1], fill='tozeroy'))
    fig2.update_layout(title=f'FFT Magnitude - Integrated Simpsons Area: {integrated_area:.4f}')
    fig2.show()

In [29]:
matlab_script()